In [0]:
#creating a csv
dbutils.fs.put('dbfs:/FileStore/data/dynamic_column.csv',"""id, name, loc, emailid. phone
1,ravi
2,ravi, banglore
3,prasad, chennai, sample@gmail.com, 988654
4,sam, Pune
""")

Wrote 110 bytes.
Out[25]: True

In [0]:
df=spark.read.csv(path='dbfs:/FileStore/data/dynamic_column.csv', header=True)
df.show()

+---+------+---------+-----------------+
| id|  name|      loc|   emailid. phone|
+---+------+---------+-----------------+
|  1|  ravi|     null|             null|
|  2|  ravi| banglore|             null|
|  3|prasad|  chennai| sample@gmail.com|
|  4|   sam|     Pune|             null|
+---+------+---------+-----------------+



In [0]:
dbutils.fs.put('dbfs:/FileStore/data/dynamic_without_column.csv',"""1,ravi
2,ravi, banglore
3,prasad, chennai, sample@gmail.com, 988654
4,sam, Pune
""")

Wrote 80 bytes.
Out[27]: True

In [0]:
df1=spark.read.csv(path='dbfs:/FileStore/data/dynamic_without_column.csv')
df1.show() #unable to read all column values, always depend on 1st row only

+---+------+
|_c0|   _c1|
+---+------+
|  1|  ravi|
|  2|  ravi|
|  3|prasad|
|  4|   sam|
+---+------+



In [0]:
# to resolve above problem, always read such data in text, not in csv
df2 = spark.read.text(paths='dbfs:/FileStore/data/dynamic_without_column.csv')
display(df2)

value
"1,ravi"
"2,ravi, banglore"
"3,prasad, chennai, sample@gmail.com, 988654"
"4,sam, Pune"


In [0]:
from pyspark.sql.functions import split, size, max

In [0]:
df2.withColumn('splitable_col', split('value',',')).show()

+--------------------+--------------------+
|               value|       splitable_col|
+--------------------+--------------------+
|              1,ravi|           [1, ravi]|
|    2,ravi, banglore|[2, ravi,  banglore]|
|3,prasad, chennai...|[3, prasad,  chen...|
|         4,sam, Pune|     [4, sam,  Pune]|
+--------------------+--------------------+



In [0]:
df2 = df2.withColumn('splitable_col', split('value',',')).drop('value')
display(df2)

splitable_col
"List(1, ravi)"
"List(2, ravi, banglore)"
"List(3, prasad, chennai, sample@gmail.com, 988654)"
"List(4, sam, Pune)"


In [0]:
df2.select('splitable_col' ,size('splitable_col')).show(truncate=False)

+-------------------------------------------------+-------------------+
|splitable_col                                    |size(splitable_col)|
+-------------------------------------------------+-------------------+
|[1, ravi]                                        |2                  |
|[2, ravi,  banglore]                             |3                  |
|[3, prasad,  chennai,  sample@gmail.com,  988654]|5                  |
|[4, sam,  Pune]                                  |3                  |
+-------------------------------------------------+-------------------+



In [0]:
df2.select(max(size('splitable_col'))).collect()[0][0]

Out[39]: 5

In [0]:
for i in range(df2.select(max(size('splitable_col'))).collect()[0][0]):
    df2=df2.withColumn('col'+str(i), df2['splitable_col'][i])
display(df2)

splitable_col,col0,col1,col2,col3,col4
"List(1, ravi)",1,ravi,null,null,null
"List(2, ravi, banglore)",2,ravi,banglore,null,null
"List(3, prasad, chennai, sample@gmail.com, 988654)",3,prasad,chennai,sample@gmail.com,988654
"List(4, sam, Pune)",4,sam,Pune,null,null


In [0]:
final_df = df2.drop('splitable_col')
display(final_df)

col0,col1,col2,col3,col4
1,ravi,null,null,null
2,ravi,banglore,null,null
3,prasad,chennai,sample@gmail.com,988654
4,sam,Pune,null,null
